In [150]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum
import gurobipy as gb

In [151]:
demand = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Final Exam\ecogreen_energy_demand.csv")
supply = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Final Exam\ecogreen_energy_supply.csv")

In [152]:
model = Model("EcoGreen Energy Expansion")

In [153]:
plants = [i for i in range(supply.shape[0])]
provinces = [j for j in range(demand.shape[0])]

In [154]:
x = model.addVars(plants, vtype=GRB.BINARY, name="Plant")
y = model.addVars(plants, provinces, vtype=GRB.CONTINUOUS, name="Energy_Supplied")

In [155]:
fixed_cost = gb.quicksum(supply.loc[i, "Fixed"] * x[i] for i in supply.index)
variable_cost = gb.quicksum(supply.loc[i, f"Province {j+1}"] * y[i, j] for i in supply.index for j in range(10))

In [156]:
model.setObjective(fixed_cost + variable_cost, GRB.MINIMIZE)

In [157]:
# Constraints

# Capacity Constraints
for i in plants:
    model.addConstr(quicksum(y[i, j] for j in provinces) <= supply.loc[i, "Capacity"] * x[i], name=f"Capacity_Plant_{i+1}")

# Demand Satisfaction Constraints
for j in provinces:
    model.addConstr(quicksum(y[i, j] for i in plants) == demand.loc[j, "Demand"], name=f"Demand_Province_{j+1}")

# Exclusive site choices
model.addConstr(x[9] + x[14] + x[19] <= 1, "MutualExclusion_10_15_20")

# Conditional site openings
model.addConstr(x[2] <= x[3], "Link_3_4")
model.addConstr(x[2] <= x[4], "Link_3_5")
model.addConstr(x[4] <= x[7] + x[8], "Dependency_5_8_9")

# Region A & B Ratio
model.addConstr(quicksum(x[i] for i in range(10)) <= 2 * quicksum(x[i] for i in range(10, 20)), "Region_Ratio")

# Minimum output from sites 1-5
min_output_sites_1_5 = quicksum(y[i, j] for i in range(5) for j in provinces)
total_output_all_sites = quicksum(y[i, j] for i in plants for j in provinces)
model.addConstr(min_output_sites_1_5 >= 0.3 * total_output_all_sites, "Min_Output_1_5")

# Maximum energy contribution constraint for each plant to each province
#for j in provinces:
#    for i in plants:
#        model.addConstr(y[i, j] <= 0.5 * demand.loc[j, "Demand"], f"Max_Contribution_Plant_{i+1}_Province_{j+1}")

<gurobi.Constr *Awaiting Model Update*>

In [158]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 36 rows, 220 columns and 650 nonzeros
Model fingerprint: 0xfad78706
Variable types: 200 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+05]
  Objective range  [2e-01, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Found heuristic solution: objective 2.879195e+08
Presolve time: 0.00s
Presolved: 36 rows, 220 columns, 500 nonzeros
Variable types: 200 continuous, 20 integer (20 binary)

Root relaxation: objective 1.685643e+08, 66 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.6856e+08    0    4 2.879

In [159]:
if model.status == GRB.OPTIMAL:
    print(f'Optimal cost: {model.ObjVal:.2f}')
    print("Plants that will be opened:")
    for i in plants:
        if x[i].X > 0.5:
            print(f"Plant {i+1}")
else:
    print("No optimal solution found.")

Optimal cost: 192980662.89
Plants that will be opened:
Plant 1
Plant 2
Plant 3
Plant 4
Plant 5
Plant 7
Plant 8
Plant 11
Plant 15
Plant 16
Plant 17


In [160]:
# Print Energy Supplied (y_ij)
print("\nEnergy Supplied (GWh):")
for i in supply.index:
    for j in demand.index:
        if y[i, j].x > 0:  # Check if energy is supplied
            print(f"Plant {i+1} to Province {j+1}: {y[i, j].x:.2f}")


Energy Supplied (GWh):
Plant 1 to Province 4: 125260.00
Plant 2 to Province 1: 134211.00
Plant 2 to Province 3: 881.00
Plant 3 to Province 7: 67681.00
Plant 3 to Province 9: 1486.00
Plant 4 to Province 6: 57834.00
Plant 5 to Province 3: 79513.00
Plant 7 to Province 3: 45112.00
Plant 7 to Province 9: 94870.00
Plant 8 to Province 7: 57981.00
Plant 11 to Province 2: 19264.00
Plant 11 to Province 7: 2043.00
Plant 11 to Province 8: 171393.00
Plant 15 to Province 4: 37883.00
Plant 15 to Province 5: 116190.00
Plant 16 to Province 9: 63684.00
Plant 16 to Province 10: 114109.00
Plant 17 to Province 2: 109494.00
Plant 17 to Province 6: 80415.00
